<a href="https://colab.research.google.com/github/Johar98/Machine-Learning-Algorithm/blob/main/RNN_Play_Geberator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [3]:
path_to_file = tf.keras.utils.get_file("shakespears.txt",
                                       "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
#read, then decode from py2 compat
text = open(path_to_file , 'rb').read().decode(encoding = 'utf-8')
#length of text is the number of character in it
print('length of text : {} character'.format(len(text)))

length of text : 1115394 character


In [5]:
#take a look at the first 250 character in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text)) #sort all the unique character in a sat
#creating a mapping from unique character to indices
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])
text_as_int = text_to_int(text)

In [7]:
#lets look at how part of our text in encoded
print("text:" , text[:13])
print("encoded:" , text_to_int(text[:13]))

text: First Citizen
encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
      pass
  return "".join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


In [9]:
seq_length = 100 #length of sequence for a training example
exmples_per_epoch = len(text)//(seq_length+1)
#create training example/targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [10]:
sequences = char_dataset.batch(seq_length+1 , drop_remainder = True)

In [11]:
def split_input_target(chunk): #for the example :hello
  input_text = chunk[:-1] #hell
  target_text = chunk[1:] #ello
  return input_text , target_text #hell , ello
dataset = sequences.map(split_input_target) # we can map to apply the above function to every entry

In [12]:
for x,y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024
#buffer size to shuffle the dataset
#(tf data is designed to work with possiby infinite sequence so it doesn't attempt to shuffle the entire sequence in memory , instead
# it maintains a buffer in which it shuffle elements)
BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE , drop_remainder = True)


In [14]:
def build_model(vocab_size , embedding_dim , rnn_units , batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size , None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful = True, recurrent_initializer="glorot_uniform"),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model
model = build_model(VOCAB_SIZE , EMBEDDING_DIM , RNN_UNITS , BATCH_SIZE)
model.summary

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fcc8d645390>>

In [15]:
for input_example_batch , target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  #ask our model for a prediction on our first batch of training data
  print(example_batch_predictions.shape , "#(batch_size , sequence_length , vocab_size)")
  #print out the output shape

(64, 100, 65) #(batch_size , sequence_length , vocab_size)


In [16]:
#we can see that the prediction is an array of 64 arrays , one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)
#let's exam one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred) #notice this is a 2d array of length 100 , where each integerior array is the prediction for the next character at each time step
#and finally well look at a predictions at the first timstep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

64
tf.Tensor(
[[[-6.62135892e-03 -4.87826439e-03 -2.44118576e-03 ...  4.56168829e-03
    2.72521051e-04 -2.05360446e-03]
  [-2.53627053e-03 -4.53067804e-03 -2.69192457e-03 ...  4.34534438e-03
   -6.92740665e-04 -1.98881282e-03]
  [-9.40695126e-03  3.63098038e-03 -6.82335673e-03 ...  6.83177495e-03
   -4.51169861e-03  6.50289934e-03]
  ...
  [-5.91002079e-03  5.66960266e-03  5.67942159e-04 ... -4.21425560e-03
   -1.23961875e-02 -5.94716286e-03]
  [-1.17876558e-02  1.04077691e-02 -3.53369419e-03 ...  1.89227052e-04
   -1.28982514e-02  1.87355373e-03]
  [-1.66081395e-02  1.41211795e-02 -6.70389552e-03 ...  3.99772683e-03
   -1.28646214e-02  7.73449335e-03]]

 [[-4.10033669e-03 -5.49616176e-04  4.88710171e-03 ... -5.77423256e-03
   -2.91904691e-03 -7.63809029e-03]
  [-1.59133971e-03 -2.90562119e-03  7.07872072e-03 ...  1.32895657e-05
    2.26457167e-04 -6.30024541e-03]
  [ 9.39406455e-05 -2.63957540e-04  4.23443969e-03 ... -3.18317930e-03
   -3.03395186e-03 -2.61153700e-03]
  ...
  [-2.799

In [17]:
#if we want to determine the predicted character we need to sample the output distribution(pick a value base on probabilities)
sampled_indices = tf.random.categorical(pred , num_samples=1)
#now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices , (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)
predicted_chars #and this is what the model predicted for training sequence 1

"X&vCoFmQHtLiSkW---s:kEy&S;wvrpp'aftOLA$3&ZkhiT?,n:jq';d.DwhyWffpTlU:tB,rVog3kf;rhywJSwgvM,Qk,eioY?XF"

In [18]:
def loss(labels , logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [19]:
model.compile(optimizer = "adam" , loss=loss)

In [20]:
#directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints" #whose we want to save
#name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir , "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix , 
    save_weights_only =True
)

In [1]:
history  = model.fit(data,epochs=40,callbacks=[checkpoint_callback])

NameError: ignored

In [ ]:
model = build_model(VOCAB_SIZE , EMBEDDING_DIM , DNN_UNITS , batch_size = 1)

In [ ]:
model.load_weight(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
def generate_text(model , start_string):
  #evaluation step (generating text using the learned model)
  #number of character to generate
  num_generate = 800
  #converting our start string to numbers(vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_sval , 0)
  #empty string to store our results
  text_generate = []
  #low temperature results in more predictable text
  #higher temperature results in more surprising text
  #experiment to find the best setting
  temperature = 1.0
  #here batch size == 1
  model.result_states()
  for i in range(unm_generate):
    predictions = model(input_eval)
    #remove the batch dimension
    predictions = tf.sequeeze(predictions , 0)
    #using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions , num_sample=1)[-1 , 0].numpy()
    #we pass the predicted characters as the next input to the model
    #along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id] , 0)
    text_generated.append(idx2char[predicted_id])
return(start_string+"".join(text_generated))
inp = input("type a starting string:")
print(generate_text(model , inp))